# CKP8087 - Estrutura de Dados
<img  src="https://img.shields.io/badge/UFC-CKP9011-000000?style=for-the-badge&logo=" /> <img src="https://img.shields.io/badge/Jupyter-000000?style=for-the-badge&logo=jupyter&logoColor=white" /> <img src="https://img.shields.io/badge/Python-000000?style=for-the-badge&logo=python&logoColor=white" />

*Vaux Gomes*

## Parte I - Preparação dos Dockers

#### Pre-requisitos
 - Docker
 - Docker Compose ou Plugin

#### Comandos

```sh
$ cd /path/to/work
$ cat > docker-compose.yml << 'EOF'
service:
  postgres:
    image: postgres
    container_name: postgres
    ports:
      - 5432:5432
    expose:
      - 5432
    networks:
      - local-network
    environment:
      - POSTGRES_USER=root
      - POSTGRES_PASSWORD=root
      - POSTGRES_DB=postgres

  neo4j:
    image: neo4j:latest
    container_name: neo4j
    ports:
      - "7474:7474"   # Neo4j Browser
      - "7687:7687"   # Bolt protocol
    networks:
      - local-network
    environment:
      - NEO4J_AUTH=neo4j/12345678             # Set do username/password
    volumes:
      - ./Neo4j/data:/data                    # Pasta para armazenar os dados
      - ./Neo4j/logs:/logs                    # Pasta para armazenar os logs
      - ./Neo4j/import:/var/lib/neo4j/import  # Pasta para importe de CSV
      - ./Neo4j/plugins:/plugins              # Pasta dos plugins
      - ./Neo4j/dumps:/dumps                  # Pasta para os dumps

networks:
  local-network:
    driver: bridge
EOF
```

<div class="alert alert-block alert-info">
    [OPCIONAL]: Escolha um nome melhor para o banco
</div>

## Parte II - Restauração do Dump

```sh
# Start do container
$ cd /path/to/work
$ docker-compose -up -d postgres

# Restore do dump
$ docker exec -i postgres psql -U root -d postgres < /path/to/backup.dump
```

![](./files/images/dbeaver.png)

## Parte III - Importando e tradandos os dados

In [1]:
# Installs - Necessário porque meu Jupyter está em um Docker
!pip install pandas  
!pip install sqlalchemy
!pip install psycopg2-binary       # Drive do postgress 

# Imports
import numpy as np
import pandas as pd
import psycopg2 as pg

In [2]:
# DB Config
username  = 'root'
password  = 'root'
host      = 'postgres' # NOME DO CONTAINER
port      = '5432'
database  = 'postgres' # NOME DO DATABASE

# Connection
conn = pg.connect(
    user=username,
    password=password,
    host=host,
    port=port,
    dbname=database
)

# Dataframe
query = '''
SELECT 
    id_member_anonymous as member,  
    id_group_anonymous as group,
    text_content_anonymous as text,
    score_sentiment, score_misinformation
FROM tb_whatsapp_messages WHERE trava_zap = false'''
df = pd.read_sql(query, conn)

conn.close()

# Info
rows = df.shape[0]

# Data
print(df.shape)
df.head()

/tmp/ipykernel_14890/364225286.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(407911, 5)


,member,group,text,score_sentiment,score_misinformation
0,eacc81d81047368e08bdcee59a0e69e2,970fc18f0d5608107b7822a2adbac3f8,None,NaN,NaN
1,542d038bf37b9f9871d6e8dac6fd4230,589e16e85b442fa82e8e0061fa2731e6,Vou ali,0.0000,NaN
2,3a8e41b9e1da548ef0acd0a57b398da4,e110071613239754d38878f7e046e95b,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867
3,3a8e41b9e1da548ef0acd0a57b398da4,7ee4235534ec624ebd61373b87ad8c20,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867
4,3a8e41b9e1da548ef0acd0a57b398da4,ee85f63c945ffa50ba8bb57acf2c1bf9,Jovem vai a sessão parlamentar na câmara dos v...,0.6371,0.001867


### Filtros & Flags

In [3]:
# Filtro do número de palavras (5+)
df = df[df.text.str.count('\s').gt(3)] # 4 espaços ou mais
print(f'Reduzido para: {df.shape} ({(df.shape[0]/rows)*100:.2f}% do tamanho original)')

Reduzido para: (186316, 5) (45.68% do tamanho original)


In [7]:
# Flag de viral
counts = df['text'].value_counts()
df['viral'] = df['text'].isin(counts[counts > 1].index)
print(f'Reduzido para: {df[df.viral == True].shape} ({(df[df.viral == True].shape[0]/rows)*100:.2f}% do tamanho)')

Reduzido para: (110091, 6) (26.99% do tamanho original)


In [9]:
# Flaf de misinformation
df['misinformation'] = df.score_misinformation >= 0.7
print(f'Reduzido para: {df[df.misinformation == True].shape} ({(df[df.misinformation == True].shape[0]/rows)*100:.2f}% do tamanho)')

Reduzido para: (8749, 8) (2.14% do tamanho)


### Grafos

In [10]:
def build(df, v=False):
    '''
    Função para mapeamento do dataframe em uma estrutura de contagem de mensagens
    
    G = {
        'sender': {
            'receiver_1': 10,
            'receiver_2': 10
        }...
    }
    '''
    
    graph = {}
    
    for g in df.group.unique()[:]:
        df_ = df[df.group == g]                 # Group filter
        members_ = df_.member.value_counts()    # Member counts
        
        for s, count in members_.items():       # s: Sender
            for r in members_.keys():           # r: Receiver
                if s == r:
                    continue
    
                graph[s] = graph.get(s, {})
                graph[s][r] = graph[s].get(r, 0) + count

    #
    members = df.member.unique()
    groups = df.group.unique()
    
    # Verbose
    if v:
        print('- Total de mensagens:', df.shape[0])
        print('- Número de membros:\n  - Ativos:', len(graph))
        print('  - Total:', len(members))
        print('- Número de grupos:', len(groups))
        print()
        
        
    return graph, members, groups


#### Grafo Geral

In [11]:
%time geral, g_members, _ = build(df, v=True)

- Total de mensagens: 186316
- Número de membros:
  - Ativos: 6149
  - Total: 6165
- Número de grupos: 237

CPU times: user 3.87 s, sys: 0 ns, total: 3.87 s
Wall time: 3.87 s


#### Grafo Viral

> Considerando viral se houver uma mensagem que foi enviada mais de uma vez

In [12]:
%time viral, v_members, _ = build(df[df['viral'] == True], v=True)

- Total de mensagens: 110091
- Número de membros:
  - Ativos: 3472
  - Total: 3487
- Número de grupos: 225

CPU times: user 2.52 s, sys: 0 ns, total: 2.52 s
Wall time: 2.52 s


#### Grafo Desinformação

In [13]:
%time misinformation, m_members, _ = build(df[df['misinformation'] == True], v=True)

- Total de mensagens: 8749
- Número de membros:
  - Ativos: 1621
  - Total: 1647
- Número de grupos: 175

CPU times: user 193 ms, sys: 1 ms, total: 194 ms
Wall time: 193 ms


## Part IV - Conexão com o Neo4J e Criação dos Grafos

```sh
# Start do container
$ cd /path/to/work
$ docker-compose -up -d neo4j
```

<div class="alert alert-block alert-info">
    A versão community do Neo4J não deixa fácil existirem vários databases, então eu joguei todos os databases em um único grafo com arestas de tipos diferentes.
</div>

In [14]:
# Installs - Necessário porque meu Jupyter está em um Docker
!pip install py2neo

# Imports
from py2neo import Graph

In [15]:
uri = "bolt://neo4j:7687"      # Deve coincidir com os valores informados no docker-compose.yml. Nesse caso o nome do serviço.
username = "neo4j"             # O Neo4j exige esse usuário
password = "12345678"          # O Neo4j exige uma senha de 8 dígitos

#
g = Graph(uri, auth=(username, password))
# session = g.session(database="neo4j") # Caso você tenha a versão paga

#### Nós

In [16]:
def create_nodes(driver, members):
    query = f'''UNWIND [
      {','.join(map(lambda m: f'{{name: "{m}"}}', members))}
    ] AS membro
    
    CREATE (m:Membro {{name: membro.name}})
    '''
    
    _ = driver.run(query)
    return 

In [17]:
# Criando os membros
%time create_nodes(g, m_members)

CPU times: user 2.16 ms, sys: 7.03 ms, total: 9.19 ms
Wall time: 245 ms


<div class="alert alert-block alert-info">
    Verificar com <code>MATCH(n) RETURN n LIMIT 50</code>
</div>

![](./files/images/neo4j-1.png)

#### Arestas

In [18]:
def create_edges(driver, graph, start=0, v=False):
    count = start
    step = len(graph)//10
    next = start + 1
    
    for s, msgs in graph.items():
        for r in msgs:
            query = f'''
                MATCH(r:Membro {{name: "{s}"}})
                MATCH(s:Membro {{name: "{r}"}})
                CREATE (r)-[e:MSGs {{total: {graph[s][r]}}}]->(s)
            '''
            _ = driver.run(query)

        # Verbose
        if v:
            count += 1
            if count == next:
                print(f'{count:04d}/{len(graph)} ~ ({count/len(graph)*100:.2f})%')
                next += step

In [19]:
# Misinformation
%time _ = create_edges(g, misinformation, v=True)

0001/1621 ~ (0.00)% 0324/1621 ~ (0.20)% 0486/1621 ~ (0.30)% 0648/1621 ~ (0.40)% 0810/1621 ~ (0.50)% 0972/1621 ~ (0.60)% 1134/1621 ~ (0.70)% 1296/1621 ~ (0.80)% 1458/1621 ~ (0.90)% 1620/1621 ~ (1.00)% CPU times: user 9.98 s, sys: 1.87 s, total: 11.8 s
Wall time: 3min 36s


<div class="alert alert-block alert-info">
    Verificar com <code>MATCH(n) RETURN n LIMIT 300</code>
</div>

![](./files/images/neo4j-3.png)

#### Backup do banco

```sh
#              CONTAINER                         DB
$ docker exec -it neo4j neo4j-admin database dump neo4j --to-path=/dumps/
```

### Parte V - Cálculos das métricas